# BIG DATA A3 - Notebook du projet

<i>Strasbourg, Juillet 2019</i>

Alors que la canicule frappe dûrement la France, que Greta Thunberg ne cesse de faire la une des journaux suite à ses déclarations alarmantes, et qu'aucun pays ne prend de réelles actions au sujet du climat, notre jeune équipe d'ingénieurs informaticiens décide de chercher une solution qui permettra d'aider à lutter contre le réchauffement climatique.

À eux 5, ils décident de réaliser un programme qui allait permettre d'optimiser les trajets des livraisons effectuées par les companies de livraisons, et ce dans le but de réduire massivement la pollution qu'elles émettent.
A l'heure actuelle, les livreurs réalisent leur itinéraire selon leur expérience, ou alors selon leurs préférences. Ces trajets ne sont donc pas optimisés, et c'est là dessus que va intervenir notre équipe.

Ce programme d'optimisation, en plus de pouvoir être utilisé par les livreurs, pourra également être utilisé pour définir le trajet des drones de livraisons, pratique de plus en plus populaire, avec notamment Amazon qui a déclaré récemment que dans les mois qui suivent des drones de livraisons allaient être déployés dans certaines villes.

# Le problème

Pour synthéthiser, le problème que notre équipe doit résoudre est le suivant :

<b>Diminuer au maximum la distance parcourue pour effectuer toutes les livraisons, tout en respectant les fenêtres de livraison de chaque ville.</b>

# Solutions

Ci-dessous seront listées chaque solution que l'équipe va apporter pour permettre de résoudre le problème d'optimisation des livraisons.


## L'algorithme de la recherche tabou
### Présentation

Dans notre contexte, la recherche tabou consiste en ce qui suit :

Partir d'une ville donnée, regarder les villes les plus proches et choisir celle qui est la plus près et dont la fenêtre de livraison est ouverte.
Les villes déjà explorées entrent dans la "liste tabou" de taille variable, et tant qu'elles seront dans cette liste le livreur n'aura 